In [1]:
import backtrader as bt
import quantstats
import akshare as ak
import efinance as ef
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import run
import sys
import math
import imgkit
from PIL import Image
from scipy import stats
import empyrical as ey
import itertools 
import collections
import datetime
import riskfolio as rp
import tradesys as ts
import math

### 1，找出所有需要的基金代码

### 2，利用akshare取基金日频净值数据

### 3，编写函数批量计算risk parity权重与min risk 权重

### 4 从结果文档中搜索数据，生成所需数据宽表

In [2]:
def str2timegap(str1 ='2019年第一季度'):
    year = str1[:4]
    if '一' in str1:
        start_md ='01-01'
        end_md ='03-31'
    if '二' in str1:
        start_md ='04-01'
        end_md ='06-30'
    if '三' in str1:
        start_md ='07-01'
        end_md ='09-30'
    if '四' in str1:
        start_md ='10-01'
        end_md ='12-31'
    startdt = year+'-'+ start_md
    enddt = year+'-'+ end_md
    return [startdt,enddt]

In [3]:
def format2result(codelist,str2 = '2019年第一季度'):
    list1=  str2timegap(str2)
    df = calculate_weight(codelist,startdate =list1[0],enddate = list1[1],objdf = df1)
    df['持仓时间'] = str2
    return df

In [4]:
def getquarter(date = '2021-01-01'):
    year = date[:4]
    i = int(date[5:7])
    if i ==1 or i ==2 or i==3:
        quarter = '01-01'
    if i ==4 or i ==5 or i==6:
        quarter = '04-01'
    if i ==7 or i ==8 or i==9:
        quarter = '07-01'
    if i ==10 or i ==11 or i==12:
        quarter = '10-01'
    result = str(year)+'-'+str(quarter)
    return result

In [5]:
def getquarter(date = '2021-01-01'):
    year = date[:4]
    i = int(date[5:7])
    if i ==1 or i ==2 or i==3:
        quarter = '01-01'
    if i ==4 or i ==5 or i==6:
        quarter = '04-01'
    if i ==7 or i ==8 or i==9:
        quarter = '07-01'
    if i ==10 or i ==11 or i==12:
        quarter = '10-01'
    result = str(year)+'-'+str(quarter)
    return result


### 5 生成文件

In [51]:
class Strategy(bt.Strategy):
    def __init__(self):
        self.order = True
        self.quarter = '2017-01-01'
        # a = self.broker.getvalue()
        # b = self.broker.getcash()
        # print(a)
        self.valuelist = {}
        
    def next(self):
        curdate = str(self.datas[0].datetime.date(0))
        self.valuelist[curdate] = self.broker.getvalue()
        if self.order == False:
            quarter = getquarter(curdate)
            if quarter != self.quarter:
                self.order = True
                self.quarter = quarter
                for d in self.datas:
                    id1 = d.params.dataname[-10:-4]
                    position1 = self.positionsbyname[str(int(id1))]
                    if position1.size < 0:
                        print(position1.size)
                        print('----------!!!!!!!!!!!!---------------')
                    if position1.size > 0:
                        self.order_target_value(d,target = 0 )
            #print(curdate, ' cash:', self.broker.get_cash(), 'value:', self.broker.getvalue())
            return

        if self.order == True:
            b = self.broker.getcash()
            for d in self.datas:
                id1 = d.params.dataname[-10:-4]
                if self.quarter in d.target.keys():
                    buyvalue = (d.target[self.quarter])
                    self.order_target_percent(d,target = buyvalue*0.98)
            self.order = False
            #print(curdate, ' cash:', self.broker.get_cash(), 'value:', self.broker.getvalue())
            return


In [52]:
def funcreturns(dfname = '股票型.xlsx',by = 'w_risk_parity'):
    df_result1 = pd.read_excel(dfname)
    df_holdhist = df_result1[['key_0', by,'持仓时间']].reset_index()
    df_holdhist['time'] = df_holdhist['持仓时间'].apply(str2timegap).apply(lambda x:x[0])#.apply(timeformat)
    ticker_names = list(set([alls for alls in df_holdhist['key_0']]))
    targets = {ticker: {} for ticker in ticker_names}
    for i in range(df_holdhist.shape[0]):
        targets[df_holdhist['key_0'][i]].update({df_holdhist['time'][i]: df_holdhist[by][i]})
        
    #####################################################################################3
    
    cerebro = bt.Cerebro()
    for ticker, target in targets.items():
        dataname_i ='datas/'+str(ticker).zfill(6)+'.csv'
        print(dataname_i)
        data = bt.feeds.GenericCSVData(
                                        dataname=dataname_i,
                                        fromdate=datetime.datetime(2017, 1, 1),
                                        todate=datetime.datetime(2021, 6, 30),
                                        nullvalue=0.0,
                                        dtformat=('%Y-%m-%d'),
                                        datetime=1,
                                        high = 2,
                                        low=2,
                                        open=2,
                                        close=2,
                                        volume=-1,
                                        openinterest=-1
                                        )
        data.target = target
        cerebro.adddata(data, name=str(ticker))

    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name = "TA")
    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name = "TR")
    cerebro.addanalyzer(bt.analyzers.SQN, _name = "SQN")
    cerebro.addanalyzer(bt.analyzers.Returns, _name = "Returns")
    cerebro.addanalyzer(bt.analyzers.TimeDrawDown, _name = "TimeDrawDown")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio', timeframe=bt.TimeFrame.Days, annualize=True, riskfreerate=0.03)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='SharpeRatio_A')
    
    cerebro.addanalyzer(bt.analyzers.Transactions, _name='Trans')

    cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Years,_name='timereturns')
    
    
    cerebro.addstrategy(Strategy)
    cerebro.broker.setcash(10000)

    # Execute
    results = cerebro.run()
    portfolio_stats = results[0].analyzers.getbyname('PyFolio')
    returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
    returns.index = returns.index.tz_convert(None)
    totalTrade = results[0].analyzers.getbyname("TA").get_analysis()
    sqn = results[0].analyzers.SQN.get_analysis()["sqn"]
    Returns = results[0].analyzers.Returns.get_analysis()
    timedrawdown = results[0].analyzers.TimeDrawDown.get_analysis()
    sharpe = results[0].analyzers.SharpeRatio.get_analysis()
    sharpeA = results[0].analyzers.SharpeRatio_A.get_analysis()

    backtest_results = pd.Series()

    backtest_results["总收益率"] = Returns["rtot"]
    backtest_results["平均收益率"] = Returns["ravg"]
    backtest_results["年化收益率"] = Returns["rnorm"]
    prepare_returns = False
    # 计算夏普比率
    if returns.std() == 0.0:
        sharpe = 0.0
    else:
        sharpe = quantstats.stats.sharpe(returns = returns, rf = 0.03)
    # 计算αβ值
    backtest_results["波动率"] = quantstats.stats.volatility(returns = returns, prepare_returns = prepare_returns)
    backtest_results["赢钱概率"] = quantstats.stats.win_rate(returns = returns, prepare_returns = prepare_returns)
    backtest_results["收益风险比"] = quantstats.stats.risk_return_ratio(returns = returns, prepare_returns = prepare_returns)
    backtest_results["夏普比率"] = sharpe


    # 最大回撤
    md = quantstats.stats.max_drawdown(prices = returns)
    backtest_results["最大回撤"] = md
    s1 = pd.Series(results[0].valuelist)
    s1.name = by
    backtest_results.name = by
    return s1,backtest_results

In [53]:
a = funcreturns(dfname = '股票型.xlsx',by = 'w_mean_risk')[0]
b = funcreturns(dfname = '股票型.xlsx',by = 'w_risk_parity')[0]
c = funcreturns(dfname = '股票型.xlsx',by = 'w_equal_weight')[0]
d = funcreturns(dfname = '股票型.xlsx',by = 'w_mean_risk')[1]
e = funcreturns(dfname = '股票型.xlsx',by = 'w_risk_parity')[1]
f = funcreturns(dfname = '股票型.xlsx',by = 'w_equal_weight')[1]

datas/160416.csv
datas/671010.csv
datas/159906.csv
datas/159909.csv
datas/540008.csv
datas/163208.csv
datas/163818.csv
datas/540010.csv
datas/121002.csv
datas/162411.csv
datas/159918.csv
datas/159915.csv
datas/162102.csv
datas/320022.csv
datas/206012.csv
datas/160416.csv
datas/671010.csv
datas/159906.csv
datas/159909.csv
datas/540008.csv
datas/163208.csv
datas/163818.csv
datas/540010.csv
datas/121002.csv
datas/162411.csv
datas/159918.csv
datas/159915.csv
datas/162102.csv
datas/320022.csv
datas/206012.csv
datas/160416.csv
datas/671010.csv
datas/159906.csv
datas/159909.csv
datas/540008.csv
datas/163208.csv
datas/163818.csv
datas/540010.csv
datas/121002.csv
datas/162411.csv
datas/159918.csv
datas/159915.csv
datas/162102.csv
datas/320022.csv
datas/206012.csv


KeyboardInterrupt: 

In [45]:
pd.concat([a,b,c],axis = 1).to_excel('股票型plotreturns.xlsx')

In [46]:
pd.concat([d,e,f],axis =1 ).to_excel('股票型result.xlsx')

In [54]:
funcreturns(dfname = '混合型.xlsx',by = 'w_min_risk')

datas/375010.csv
datas/020023.csv
datas/519110.csv
datas/320007.csv
datas/110025.csv
datas/400015.csv
datas/398041.csv
datas/240022.csv
datas/519095.csv
datas/377240.csv
datas/162201.csv
datas/162202.csv
datas/290012.csv
datas/519005.csv


(2017-01-03    10000.0000
 2017-01-04    10000.0000
 2017-01-05    10033.1730
 2017-01-06     9984.1942
 2017-01-09    10045.4610
                  ...    
 2021-06-23    16861.6732
 2021-06-24    16792.8350
 2021-06-25    17067.0064
 2021-06-28    17206.2530
 2021-06-29    17262.6512
 Name: w_min_risk, Length: 1095, dtype: float64,
 总收益率     0.545960
 平均收益率    0.000499
 年化收益率    0.133880
 波动率      0.213277
 赢钱概率     0.520969
 收益风险比    0.043842
 夏普比率     0.557362
 最大回撤    -0.252396
 Name: w_min_risk, dtype: float64)

In [55]:
a = funcreturns(dfname = '混合型.xlsx',by = 'w_min_risk')[0]
b = funcreturns(dfname = '混合型.xlsx',by = 'w_risk_parity')[0]
c = funcreturns(dfname = '混合型.xlsx',by = 'w_equal_weight')[0]
d = funcreturns(dfname = '混合型.xlsx',by = 'w_min_risk')[1]
e = funcreturns(dfname = '混合型.xlsx',by = 'w_risk_parity')[1]
f = funcreturns(dfname = '混合型.xlsx',by = 'w_equal_weight')[1]

datas/375010.csv
datas/020023.csv
datas/519110.csv
datas/320007.csv
datas/110025.csv
datas/400015.csv
datas/398041.csv
datas/240022.csv
datas/519095.csv
datas/377240.csv
datas/162201.csv
datas/162202.csv
datas/290012.csv
datas/519005.csv
datas/375010.csv
datas/020023.csv
datas/519110.csv
datas/320007.csv
datas/110025.csv
datas/400015.csv
datas/398041.csv
datas/240022.csv
datas/519095.csv
datas/377240.csv
datas/162201.csv
datas/162202.csv
datas/290012.csv
datas/519005.csv
datas/375010.csv
datas/020023.csv
datas/519110.csv
datas/320007.csv
datas/110025.csv
datas/400015.csv
datas/398041.csv
datas/240022.csv
datas/519095.csv
datas/377240.csv
datas/162201.csv
datas/162202.csv
datas/290012.csv
datas/519005.csv
datas/375010.csv
datas/020023.csv
datas/519110.csv
datas/320007.csv
datas/110025.csv
datas/400015.csv
datas/398041.csv
datas/240022.csv
datas/519095.csv
datas/377240.csv
datas/162201.csv
datas/162202.csv
datas/290012.csv
datas/519005.csv
datas/375010.csv
datas/020023.csv
datas/519110.c

In [56]:
pd.concat([a,b,c],axis = 1).to_excel('混合型plotreturns.xlsx')

In [57]:
pd.concat([d,e,f],axis =1 ).to_excel('混合型result.xlsx')